# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder).

In [1]:
model_patch_size = 8
verbose = 0
reset_kernel = False

In [2]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedInputStream size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.BufferedOutputStream size changed, may indicate binary 


Octave is ready <oct2py.core.Oct2Py object at 0x7fed9faa1e70>

















In [3]:
import torch
torch.cuda.set_device(1)

## Config parameters
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [4]:
config, job_type = get_artifact_config_embeddings(verbose = 0)

In [5]:
dvats.config.show_attrdict(config)

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/moment-small-embedding:latest
input_ar: None
cpu: False


### Show configuration artifact

In [6]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/moment-small-embedding:latest
input_ar: None
cpu: False


## Build W&B artifact

In [7]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03a_embeddings"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03a_embeddings


In [8]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


## Get trained model artifact

### Build artifact selector
> Botch to use artifacts offline

In [9]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [10]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [11]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb: Downloading large artifact moment-small-embedding:latest, 144.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [12]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'toy:v2'

In [13]:
dvats.config.show_attrdict(enc_run.config)

r: 0.71
w: 30
freq: 1s
alias: toy
epochs: 100
mvp_ws: [10, 30]
stride: 1
time_col: None
data_cols: []
mask_sync: False
use_wandb: True
batch_size: 32
csv_config: {}
data_fpath: ~/data/toy.csv
valid_size: 0.2
mask_future: False
wandb_group: None
analysis_mode: online
artifact_name: toy
mask_stateful: True
norm_by_sample: False
train_artifact: mi-santamaria/deepvats/toy:latest
valid_artifact: None
norm_use_single_batch: False
norm_use_by_single_batch: [False]


### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [14]:
enc_run.config['batch_size']

32

In [15]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'toy:v2'

In [16]:
df = input_ar.to_df()
df.head()

wandb:   1 of 1 files downloaded.  


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


In [17]:
df.shape

(550, 3)

In [18]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(521, 3, 30)

In [19]:
timer = ut.Time()
timer.start()

1729518354.380352

In [20]:
config.enc_artifact

'mi-santamaria/deepvats/moment-small-embedding:latest'

In [21]:
print(enc_learner)

MOMENTPipeline(
  (normalizer): RevIN()
  (tokenizer): Patching()
  (patch_embedding): PatchEmbedding(
    (value_embedding): Linear(in_features=8, out_features=512, bias=False)
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (

In [22]:
enc_learn_class = str(enc_learner.__class__)[8:-2]

match enc_learn_class:
    case "momentfm.models.moment.MOMENTPipeline":
        get_embs_kwargs = {
            "cpu": config.cpu,
            "to_numpy": True,
            "verbose": 1
        }
    case "fastai.learner.Learner":
        get_embs_kwargs = {
            "stride": enc_run.config['stride'],
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 1
        }
    case "uni2ts.model.moirai.module.MoiraiModule":
        get_embs_kwargs = {
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 1,
            "patch_size": 8, #Modificar en config (añadir en base.yml & modificar lectura a "si existe, añadir"),
            "size": "small", #Modificar en config (añadir en base.yml & modificar lectura a "si existe, añadir"),
            "time": True
        }
    case _:
        print(f"Model embeddings implementation is not yet implemented for {enc_learn_class}.")

In [23]:
enc_learn_class

'momentfm.models.moment.MOMENTPipeline'

In [24]:
enc_learn_class = str(enc_learner.__class__)[8:-2]
enc_learn_class

'momentfm.models.moment.MOMENTPipeline'

In [25]:
match enc_learn_class:
    case "momentfm.models.moment.MOMENTPipeline":
        get_embs_kwargs = {
            "batch_size": enc_input.shape[0],
            "cpu": config.cpu,
            "to_numpy": True,
            "verbose": 1,
            "padd_step":2
        }
    case "fastai.learner.Learner":
        get_embs_kwargs = {
            "stride": enc_run.config['stride'],
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 1
        }
    case "uni2ts.model.moirai.module.MoiraiModule":
        get_embs_kwargs = {
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 2,
            "patch_size": model_patch_size, #Modificar en config (añadir en base.yml & modificar lectura a "si existe, añadir"),
            "time": True
        }
    case _:
        print(f"Model embeddings implementation is not yet implemented for {enc_learn_class}.")
print(f"Enc learn class {enc_learn_class}\nkwargs: {get_embs_kwargs}")

Enc learn class momentfm.models.moment.MOMENTPipeline
kwargs: {'batch_size': 521, 'cpu': False, 'to_numpy': True, 'verbose': 1, 'padd_step': 2}


In [26]:
losses, eval_results_pre, eval_results_post, t_shots, t_shot, t_evals, t_eval = fine_tune_moment_(
    X                             = df,
    enc_learn                     = enc_learner, 
    stride                        = 1,      
    batch_size                    = 128,
    cpu                           = True, 
    to_numpy                      = True, 
    verbose                       = 1, 
    time_flag                     = True,
    #n_windows                     = 32,
    n_windows_percent             = 0.8,# Enmascaro el 75%
    training_percent              = 0.1, # Entreno con el 10% de los datos
    validation_percent            = 0.3, # Evalúo con el 20% de los datos
    num_epochs                    = 10,
    shot                          = True,
    eval_pre                      = True,
    eval_post                     = True,
    lr_scheduler_flag             = False,
    lr_scheduler_name             = False,
    lr_scheduler_num_warmup_steps = 0,
    window_sizes                  = None,
    n_window_sizes                = 3,
    window_sizes_offset           = 0.05,
    windows_min_distance          = 5
)

--> fine_tune_moment_
fine_tune_moment_ | X not-windowed dataset
fine_tune_moment_ | X not-windowed dataset | Selecting Fourier's dominant frequences
Find_dominant_window_sizes_list | Find and return valid window_sizes | ... 2b ... 3
Find_dominant_window_sizes_list | Find and return valid window_sizes | ... 2b ... 3
Find_dominant_window_sizes_list | Find and return valid window_sizes | ... 2b ... 3
fine_tune_moment_ | X not-windowed dataset | Selecting Fourier's dominant frequences | [17, 10, 23]
fine_tune_moment_ | Building the datasets
fine_tune_moment_ | Processing 3 datasets
fine_tune_moment_ | Processing wlen 3
--> fine_tune_moment_single


  0%|          | 0/2 [00:00<?, ?it/s]

[] Start: 1729518354.4853218 | End: 1729518367.8710852 | Duration: 13.38576340675354 seconds


  0%|          | 0/10 [00:00<?, ?it/s]

[] Start: 1729518367.8719661 | End: 1729518414.2818763 | Duration: 46.40991020202637 seconds


...

[] Start: 1729518467.3330224 | End: 1729518480.3617313 | Duration: 13.028708934783936 seconds
fine_tune_moment_single | Evaluation summary
Eval post: 
mse: {'mse': 0.42728006642069544}
rmse: {'mse': 0.18502132034228738}
mae: {'mae': 0.38626829032702387}
smape: {'smape': 1.6540721054281655}
fine_tune_moment_single -->
fine_tune_moment_ | Processing wlen 3
--> fine_tune_moment_single


  0%|          | 0/10 [00:00<?, ?it/s]

[] Start: 1729518480.367683 | End: 1729518532.3196554 | Duration: 51.95197248458862 seconds


  0%|          | 0/2 [00:00<?, ?it/s]

[] Start: 1729518532.3208475 | End: 1729518545.8650477 | Duration: 13.54420018196106 seconds
fine_tune_moment_single | Evaluation summary
Eval post: 
mse: {'mse': 0.38556588725542495}
rmse: {'mse': 0.15726681668844006}
mae: {'mae': 0.344885628245667}
smape: {'smape': 1.366923708013195}
fine_tune_moment_single -->


In [27]:
eval_results_post

[{'mse': {'mse': 0.40897399885583696},
  'rmse': {'mse': 0.1743835939649953},
  'mae': {'mae': 0.3675627856552891},
  'smape': {'smape': 1.503446818938302}},
 {'mse': {'mse': 0.42728006642069544},
  'rmse': {'mse': 0.18502132034228738},
  'mae': {'mae': 0.38626829032702387},
  'smape': {'smape': 1.6540721054281655}},
 {'mse': {'mse': 0.38556588725542495},
  'rmse': {'mse': 0.15726681668844006},
  'mae': {'mae': 0.344885628245667},
  'smape': {'smape': 1.366923708013195}}]

In [28]:
print(lossess)
print(eval_results_pre)
print(eval_results_post)
print(t_shots)
print(t_shot)
print(t_evals)
print(t_eval)

NameError: name 'lossess' is not defined

In [ ]:
losses, eval_results_pre, eval_results_post, t_shot, t_eval_1, t_eval_2 = fine_tune_moment_(
    X                       = enc_input,
    enc_learn               = enc_learner, 
    stride                  = 1,      
    batch_size              = 128,
    cpu                     = True, 
    to_numpy                = True, 
    verbose                 = 0, 
    time_flag               = True,
    #n_windows                = 32,
    n_windows_percent       = 0.8, # Enmascaro el 75%
    training_percent        = 0.1, # Entreno con el 10% de los datos
    validation_percent      = 0.3, # Evalúo con el 20% de los datos
    num_epochs              = 10,
    shot                    = True,
    eval_pre                = True,
    eval_post               = True,
    lr_scheduler_flag       = False
)

In [ ]:
print(losses)
print(eval_results_pre)
print(eval_results_post)
print(t_shot)
print(t_eval_1)
print(t_eval_2)

In [ ]:
embs = get_enc_embs_set_stride_set_batch_size(
    X          = enc_input, 
    enc_learn  = enc_learner, 
    stride     = enc_run.config['stride'],
    **get_embs_kwargs
)

In [ ]:
enc_learner.task_name

In [ ]:
timer.end()
timer.show()

In [ ]:
#| export
if verbose > 0: print("Execution ended")
from dvats.imports import beep
beep(1)
beep(1)
beep(1)
beep(1)
beep(1)

In [ ]:
#| hide
if reset_kernel:
    import os
    os._exit(00)